In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\ChickenDisease\\Chicken-Disease\\research'

In [3]:
os.chdir("D:\Projects\ChickenDisease\Chicken-Disease")

In [4]:
%pwd

'D:\\Projects\\ChickenDisease\\Chicken-Disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path
# dataclasses are immutable, so they can't be changed
@dataclass(frozen=True) # frozen=True means that the dataclass can't be changed
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
            

            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)

            create_directories([self.config.artifacts_root]) # artifacts_root returns value from configbox in read_yaml
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_url= config.source_url,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config=DataIngestionConfig): 
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} to {self.config.local_data_file} with following info:\n{headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size : {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file to the data directory
        Function returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file) as zip_ref:
            zip_ref.extractall(unzip_path) 

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2023-09-07 08:13:00,192:INFO: common:yaml file : config\config.yaml loaded successfully]
[2023-09-07 08:13:00,195:INFO: common:yaml file : params.yaml loaded successfully]
[2023-09-07 08:13:00,197:INFO: common:Directory at : artifacts created successfully]
[2023-09-07 08:13:00,200:INFO: common:Directory at : artifacts/data_ingestion created successfully]
[2023-09-07 08:13:01,745:INFO: 430427877:Downloaded artifacts/data_ingestion/ChickenDiseaseDataset to artifacts/data_ingestion/ChickenDiseaseDataset with following info:
Server: GitHub.com
Date: Thu, 07 Sep 2023 04:13:02 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"cb9bf87e9b1baf9faf166b9ec7e9baea"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-down

BadZipFile: File is not a zip file